# Check Data Quality of the Files

checking data quality of the files
e.g. if there are redundant files or empty files, etc.

Steps:
- importing Boto because files are on S3

In [1]:
#!pip3 install s3fs boto3 mock --use-feature=2020-resolver
# there is a bug in the feature resolver, that the installation runs forever, even though the installation succeeded.
# you may have to interrupt the kernel and then s3fs is isntalled properly

^C


In [5]:
import boto3
import pandas as pd

In [16]:
def file_list(bucket_name):
    """A function that lists all files (objects) in a given bucket (argument bucket_name) and puts it in a list"""
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)
    return [i.key for i in my_bucket.objects.all()]

In [17]:
file_list('dsti-ml-nlp-class')

['Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (2).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (3).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (4).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT.csv',
 'Data/MR_TAPIYA_JOHN_BANK_STATEMENT-part2.csv',
 'Data/Mr_Tochukwu_statement-part2.csv',
 'Data/mr_gani_olalekan_statement_July_9.xlsx']

In [8]:
def create_s3_file_uri(bucket_name, file_name):
    """takes a bucket name and filename as argument and forms an s3 uri from it."""
    s3_file_uri = "s3://" + bucket_name + "/" + file_name
    return s3_file_uri

In [9]:
def file_to_pandas(s3_file_uri):
    """put a given file into a pandas dataframe. Can differentiate between .xlsx and .csv"""
    if (s3_file_uri.endswith(".csv")):
        data = pd.read_csv(s3_file_uri)
    elif (s3_file_uri.endswith(".xlsx")):
        data = pd.read_excel(s3_file_uri)
    else:
        print("This Function can handle only .xlsx or .csv files")
        data = None
    return data

In [10]:
create_s3_file_uri("dsti-ml-nlp-class", "Data/mr_gani_olalekan_statement_July_9.xlsx")

's3://dsti-ml-nlp-class/Data/mr_gani_olalekan_statement_July_9.xlsx'

In [11]:
s3_test_uri = create_s3_file_uri("dsti-ml-nlp-class", "Data/mr_gani_olalekan_statement_July_9.xlsx")

In [55]:
df = file_to_pandas(s3_test_uri)

In [52]:
def clean_dataframe (df):
    """takes a Dataframe as input, selects the column as header, which has TXN DATE in first column
    select only the columns that are not nan"""
    
    #selecting only rows with txn data and after
    tmp_df = df[list(df[df.iloc[:,0]=="TXN DATE"].index)[0]:]
    
    #removing first two lines of the dataframe and taking the third column as headers
    df_with_headers = pd.DataFrame(tmp_df)
    df_with_headers.columns = tmp_df.iloc[0].values
    df_with_headers.drop(df_with_headers.index[0], inplace=True)
    
    #reducing into a new df that has no nan columns
    df_reduced = df_with_headers[[el for el in test_data.iloc[0].values if isinstance(el, str)]]
    return df_reduced

In [56]:
clean_dataframe(df)

,TXN DATE,VAL DATE,REMARKS,DEBIT,CREDIT,BALANCE
3,2021-05-16 00:00:00,2021-05-16 00:00:00,OUTWARD (NIP) TRANSFER ONLINE TO: UNITED BANK ...,150000,NaN,3.63952e+06
4,2021-05-16 00:00:00,2021-05-16 00:00:00,COMMISSION ONLINE TO: UNITED BANK\nFOR AFRICA|...,50,NaN,3.63947e+06
5,2021-05-16 00:00:00,2021-05-16 00:00:00,VAT ONLINE TO: UNITED BANK FOR AFRICA|GANNI-OL...,3.75,NaN,3.63947e+06
6,2021-05-17 00:00:00,2021-05-17 00:00:00,COMMISSION ONLINE TO: ACCESS\nBANK|OLOLADE OMO...,25,NaN,3.59644e+06
7,2021-05-17 00:00:00,2021-05-17 00:00:00,OUTWARD (NIP) TRANSFER ONLINE TO: ACCESS BANK|...,43000,NaN,3.59647e+06
...,...,...,...,...,...,...
141,2021-06-04 00:00:00,2021-06-04 00:00:00,STAMP DUTY CHARGE FROM UBA/ GANNI- OLA SAFES A...,50,NaN,1.2585e+07
142,2021-06-05 00:00:00,2021-06-05 00:00:00,OUTWARD (NIP) TRANSFER ONLINE TO: UNITED BANK ...,510000,NaN,1.2075e+07
143,2021-06-05 00:00:00,2021-06-05 00:00:00,COMMISSION ONLINE TO: UNITED BANK FOR AFRICA|E...,50,NaN,1.2075e+07
144,2021-06-05 00:00:00,2021-06-05 00:00:00,VAT ONLINE TO: UNITED BANK FOR AFRICA|EZEOBIOR...,3.75,NaN,1.2075e+07
